In [5]:
import tensorflow as tf
import numpy as np

# Define your RNN model
class ContinuousLearningRNN(tf.keras.Model):
    def __init__(self):
        super(ContinuousLearningRNN, self).__init__()
        self.rnn_layer = tf.keras.layers.SimpleRNN(units=128, activation='relu')
        self.output_layer = tf.keras.layers.Dense(units=num_classes, activation='softmax')

    def call(self, inputs):
        rnn_output = self.rnn_layer(inputs)
        output = self.output_layer(rnn_output)
        return output

# Generate synthetic dataset
def generate_data(num_tasks, task_size):
    x_train_list, y_train_list = [], []
    for _ in range(num_tasks):
        x_train = np.random.randn(task_size, time_steps, input_size)
        y_train = np.random.randint(num_classes, size=task_size)
        x_train_list.append(x_train)
        y_train_list.append(y_train)
    return x_train_list, y_train_list

# Elastic Weight Consolidation (EWC) importance estimation
def compute_importance(prev_task_weights, x_prev_task_data, y_prev_task_labels):
    importance = []
    for i, weight in enumerate(prev_task_weights):
        # Compute importance based on Fisher Information Matrix or other methods
        # Here, we calculate the squared difference between previous and current weights
        importance.append(tf.square(prev_task_weights[i] - weight))
    return importance

# Initialize hyperparameters
num_tasks = 5
num_epochs = 10
input_size = 10
num_classes = 5
time_steps = 1  # Adding a time step dimension

# Generate synthetic dataset
x_train_list, y_train_list = generate_data(num_tasks, task_size=100)

# Initialize the model
model = ContinuousLearningRNN()

# Define optimizer and loss function
optimizer = tf.keras.optimizers.Adam()
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()

# Training loop
for task in range(num_tasks):
    # Prepare data for the current task
    x_train = x_train_list[task]
    y_train = y_train_list[task]

    # Training on the current task
    for epoch in range(num_epochs):
        with tf.GradientTape() as tape:
            # Forward pass
            logits = model(x_train)
            loss_value = loss_fn(y_train, logits)

        # Compute gradients
        grads = tape.gradient(loss_value, model.trainable_variables)

        # Update weights
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

    # Preserve knowledge using Elastic Weight Consolidation (EWC)
    if task > 0:
        # Compute importance weights based on previous task
        prev_task_weights = model.get_weights()
        x_prev_task_data = x_train_list[task-1]
        y_prev_task_labels = y_train_list[task-1]
        importance = compute_importance(prev_task_weights, x_prev_task_data, y_prev_task_labels)
        model.set_weights(prev_task_weights)  # Reset model weights to previous task weights

        # Regularize the current task based on importance
        for epoch in range(num_epochs):
            with tf.GradientTape() as tape:
                # Forward pass
                logits = model(x_train)
                loss_value = loss_fn(y_train, logits)

                # Compute regularization term based on importance
                reg_term = 0.0
                for i, weight in enumerate(model.trainable_variables):
                    reg_term += tf.reduce_sum(importance[i] * tf.square(prev_task_weights[i] - weight))

                loss_value += reg_term

            # Compute gradients
            grads = tape.gradient(loss_value, model.trainable_variables)

            # Update weights
            optimizer.apply_gradients(zip(grads, model.trainable_variables))

    # Optionally, save the model weights after each task
    model.save_weights(f"task_{task}_weights.h5")
